## Run #0 - Baseline Case

In [14]:
# dependencies

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pulp import *

# data inputs
# load time series data
data = pd.read_csv('./all-series.csv')
data = data.drop(data.columns[168:])

data = data.set_index('Hour', drop=True)
sma = data['Consumption Sma (kWh/h)'] * 1.2              # time series with electricity consumption
saft = data['Consumption Saft (kWh/h)'] * 1.05
oe_cons = data['DH plant Consumption (kWh/h)'] * 1.05

cons = sma + saft + oe_cons

hourly_price = data['Day Ahead (EUR/MWh)'] / 1000      # time series with electricity spot price

# network tariff
net_charge = 0.04257    # eur/kWh

# displacements to be run:
#   - Mar 21st, 2022 - init_disp = 385
#   - Jun 20th, 2022 - init_disp = 476
#   - Sep 19th, 2022 - init_disp = 567
#   - Dec 19th, 2022 - init_disp = 658
init_disp = 0 # no of days from march 1st, 2021 initial time step is desired

init = 0 + init_disp * 24
n = len(cons)

# data for optimization
load = np.array(cons[init:init+n])
price = np.array(hourly_price[init:init+n])



In [16]:
# create prob
prob = LpProblem('GridPVBatt', LpMinimize)

# dec vars
# energy from grid
E_G = [LpVariable('E_G_{}'.format(i), 0, None) for i in range(n)]

# obj function: min total elec cost 
# - assume price = 0 for pv electricity
# - assume price = deg_price per kWh charge/discharged
# - assume price of curtailment = spot price (not realistic)
# - assume price of sold energy from batt = spot price
# - assume price of sold energy from batt = spot price
prob += lpSum([  E_G[t] *           (price[t] + net_charge)] 
               for t in range(n))

# constraints

for t in range(0, n):

    # power balance
    prob += E_G[t] == load[t]


# solve
status = prob.solve()
print(LpStatus[status])

# make records to prep df
res = []
for t in range(n):
    record = {  'Hour': t,
                'Load': load[t],
                # 'Spot price': price[t],
                'Energy from grid': E_G[t].varValue,
    }
    res.append(record)

df = pd.DataFrame.from_records(res)
df.set_index('Hour', inplace=True)
df = df.round(2)
# print(df.to_string())
df.to_csv('00-results.csv', encoding='utf-8')

print(f'~~~~~~~~~~~~~~~\nInstallation summary')
print(f'Minimum cost of electricity: {pulp.value(prob.objective) : 0.1f}€')

# df.plot(figsize=[15,5])
# plt.show()

Optimal
~~~~~~~~~~~~~~~
Installation summary
Minimum cost of electricity:  8495985.7€
